In [ ]:
import pandas as pd
import numpy as np
import rasterio
import os
import libpysal as ps
from esda.moran import Moran

Functions for replacing NaNs with average of nearest neighbouring non-NaN raster values:

In [ ]:
# Function to calculate the average of nearest neighboring non-NaN raster cells
def interp_average(array, row, col):
    # Dimensions of the array
    rows, cols = array.shape
    total = 0
    count = 0
    for i in range(row - 1, row + 2):
        for j in range(col - 1, col + 2):
            if 0 <= i < rows and 0 <= j < cols:
                if not np.isnan(array[i, j]):
                    total += array[i, j]
                    count += 1
    if count == 0: # all neighbours are NaNs
        non_nan_inds = np.argwhere(~np.isnan(array))
        # Calculate Euclidean distances to non-NaN indices
        distances = np.linalg.norm(non_nan_inds - (row, col), axis=1)
        # Find the index of the nearest non-NaN value
        nearest_idx = non_nan_inds[np.argmin(distances)]
        return array[nearest_idx[0], nearest_idx[1]]
    else:
        return total / count
    
# Function to replace NaNs with the average of nearest neighboring non-NaN raster cells
def replace_nans(array):
    inds = list(np.argwhere(np.isnan(array)))
    print(inds)
    for ind in inds: 
        array[ind[0],ind[1]] = interp_average(array, ind[0], ind[1])

Function for computing all three variants of 3D CSC metrics:

In [ ]:

def Compute_Vars(dir_path, metric, res):
    
    files = [file for file in os.listdir(dir_path) if file.endswith(f"{metric}_{res}m.tif")]
    # Dataframe for storing metric values
    res_df = pd.DataFrame(columns=['Plot', f'{metric}_var_v1', f'{metric}_var_v2', f'{metric}_MoranI'])
    for i, f_name in enumerate(files):
        tif_file = os.path.join(dir_path, f_name)
        with rasterio.open(tif_file) as src:
            arr = src.read()
        arr = np.squeeze(arr)
        res_df.at[i, 'Plot'] = f_name[6:8] # append plot number

        # Handling empty raster cells (no hits)
        nan_count = np.sum(np.isnan(arr))
        if nan_count>0:
            # For Entropy Variability and Percent Hits Above Mean Height Variability, set NaNs to 0
            if metric in ("zent", "pzabovem"):
                arr = np.nan_to_num(arr, nan=0.0)
            # For other 3D CSC metrics, replace NaNs with interpolated value
            else:
                replace_nans(arr)

        # v1: Overall variance of 2D grid
        res_v1 = np.std(arr)
        res_df.at[i, f'{metric}_var_v1'] = res_v1

        # v2: average of means of stds along X and Y 
        std_x = np.std(arr, axis=1)
        std_y = np.std(arr, axis=0)
        mean_x = np.mean(std_x)
        mean_y = np.mean(std_y)
        res_v2 = (mean_x + mean_y)/2
        res_df.at[i, f'{metric}_var_v2'] = res_v2

        # Compute Moran's I
        weights = ps.weights.lat2W(arr.shape[1], arr.shape[0], rook=False)
        moran = Moran(arr.flatten(), weights)
        res_df.at[i, f'{metric}_MoranI'] = moran.I

    return res_df